## Import

In [1]:
from sklearn import preprocessing
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns

Using TensorFlow backend.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image
from keras.models import Sequential
# To initialize neural network
from keras.layers import Convolution2D
# Images are two dimensional, concolution step
from keras.layers import MaxPooling2D
# Pooling step
from keras.layers import Flatten
# Convert pools feature map into this large feature vector
from keras.layers import Dense
#To add fully connected layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import decode_predictions
from keras.applications.xception import Xception
#from keras.applications.InceptionResNetV2 import InceptionResNetV2
from keras import applications
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
# To initialize neural network
from keras.layers import Convolution2D
# Images are two dimensional, concolution step
from keras.layers import MaxPooling2D
# Pooling step
from keras.layers import Flatten
# Convert pools feature map into this large feature vector
from keras.layers import Dense
#To add fully connected layers
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.resnet50 import ResNet50
from keras.applications import resnet50
from keras.models import Model
import keras
from keras.optimizers import Adam
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout

from keras.callbacks import LearningRateScheduler

import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV

## Data Preparation

In [3]:
SIZE = 224
TRAIN_DIR= 'imagesdata/train/'
train_images = []
train_labels = [] 
for img_label in (os.listdir(TRAIN_DIR)):
    img_dir=os.path.join(TRAIN_DIR,img_label)
    for img_name in (os.listdir(img_dir)):
        img_path=os.path.join(img_dir,img_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(img_label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)
SIZE = 224
TEST_DIR= 'imagesdata/test/'
test_images = []
test_labels = [] 
for img_label in (os.listdir(TEST_DIR)):
    img_dir=os.path.join(TEST_DIR,img_label)
    for img_name in (os.listdir(img_dir)):
        img_path=os.path.join(img_dir,img_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(img_label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

#########################################################
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets (already split but assigning to meaningful convention)

x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

###################################################################
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [4]:
print(y_test_one_hot[150:200])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [7]:
#IMAGE_SIZE = 224
pretrained_model =applications.Xception(input_shape=(224,224, 3), include_top=False)
pretrained_model.trainable = False

model = Sequential([
    pretrained_model,
    Flatten(),
    Dense(2, activation='softmax')
])

In [8]:
def define_model(learning_rate=0.01, momentum=0.1, activation='relu', init_weights='uniform', optimizer='Adam'):   
    pretrained_model =applications.Xception(input_shape=(224,224, 3), include_top=False)
    pretrained_model.trainable = False

    model = Sequential([
    pretrained_model,
    Flatten(),
    Dense(2, activation='softmax')
    ])

    
    # compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,      
                  metrics=['accuracy'])
    return model

In [9]:
#Take a subset of train for grid search. Let us take 10% for now
from sklearn.model_selection import train_test_split
x_grid, x_not_use, y_grid, y_not_use = train_test_split(x_train, y_train, test_size=0.9, random_state=42)


from keras.wrappers.scikit_learn import KerasClassifier
batch_size = 32
epochs = 10

model = KerasClassifier(build_fn=define_model, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)

learning_rate = [0.001]
momentum = [0.9]

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3

init_weights = ['uniform']
#Also try lecun_uniform, he_normal, glorot_normal, etc. 
optimizer = ['Adam']

param_grid = dict( init_weights=init_weights, optimizer=optimizer,learning_rate=learning_rate, momentum=momentum)

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=3) 87.


.



grid_result = grid.fit(x_grid, y_grid)

# summarize results+
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/10
123/123 [==============================] - 44s 355ms/step - loss: 3.0422 - accuracy: 0.5772
Epoch 2/10
123/123 [==============================] - 41s 333ms/step - loss: 2.3570 - accuracy: 0.5691
Epoch 3/10
123/123 [==============================] - 39s 319ms/step - loss: 1.1648 - accuracy: 0.7724
Epoch 4/10
123/123 [==============================] - 39s 320ms/step - loss: 0.3634 - accuracy: 0.9431
Epoch 5/10
123/123 [==============================] - 42s 340ms/step - loss: 0.2909 - accuracy: 0.9187
Epoch 6/10
123/123 [==============================] - 39s 320ms/step - loss: 0.0381 - accuracy: 0.9837
Epoch 7/10
123/123 [==============================] - 40s 322ms/step - loss: 0.0700 - accuracy: 0.9837
Epoch 8/10
123/123 [==============================] - 40s 323ms/step - loss: 0.0083 - accuracy: 0.9919
Epoch 9/10
123/123 [==============================] - 40s 324ms/step - loss: 1.6665e-04 - accuracy: 1.0000
Epoch 10/10
62/62 [==============================] - 24s 382ms/step
E

C:\Users\banik\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/10
185/185 [==============================] - 62s 334ms/step - loss: 4.1779 - accuracy: 0.4973
Epoch 2/10
185/185 [==============================] - 59s 320ms/step - loss: 1.8327 - accuracy: 0.6919
Epoch 3/10
185/185 [==============================] - 59s 317ms/step - loss: 0.5884 - accuracy: 0.8432
Epoch 4/10
185/185 [==============================] - 60s 322ms/step - loss: 0.2809 - accuracy: 0.9459
Epoch 5/10
185/185 [==============================] - 60s 325ms/step - loss: 0.1979 - accuracy: 0.9676
Epoch 6/10
185/185 [==============================] - 59s 320ms/step - loss: 0.0657 - accuracy: 0.9838
Epoch 7/10
185/185 [==============================] - 59s 318ms/step - loss: 0.0899 - accuracy: 0.9730
Epoch 8/10
185/185 [==============================] - 60s 322ms/step - loss: 0.1424 - accuracy: 0.9676
Epoch 9/10
185/185 [==============================] - 59s 317ms/step - loss: 0.0162 - accuracy: 0.9946
Epoch 10/10
185/185 [==============================] - 59s 317ms/step - l

In [31]:
from keras.wrappers.scikit_learn import KerasClassifier
batch_size = 32
epochs = 10

model = KerasClassifier(build_fn=define_model, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)

learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.3, 0.5, 0.7, 0.9]

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
param_grid = dict(learning_rate=learning_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=16, cv=3)

grid_result = grid.fit(x_grid, y_grid)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

ValueError: learning_rate is not a legal parameter

In [11]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean = %f (std=%f) with: %r" % (mean, stdev, param))

Best: 0.502703 using {'init_weights': 'uniform', 'learning_rate': 0.001, 'momentum': 0.9, 'optimizer': 'Adam'}
Mean = 0.502703 (std=0.103668) with: {'init_weights': 'uniform', 'learning_rate': 0.001, 'momentum': 0.9, 'optimizer': 'Adam'}
